# Predição de evasão no ensino superior
#### Obs.: dataset de caso analogo de aprovação no ensino médio

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Carregando datase

In [2]:
dados_estudantes = pd.read_csv("student-data.csv")
print("Dados carregados com sucesso!")

Dados carregados com sucesso!


In [3]:
n_estudantes, n_features = dados_estudantes.shape
n_passed, _ = dados_estudantes[dados_estudantes['passed'] == 'yes'].shape
n_failed, _ = dados_estudantes[dados_estudantes['passed'] == 'no'].shape
indice_aprovacao = (n_passed / n_estudantes) * 100

print("Número de estudantes: {}".format(n_estudantes))
print("Número de features: {}".format(n_features))
print("Número de estudantes aprovados: {}".format(n_passed))
print("Número de estudantes reprovados: {}".format(n_failed))
print("Indice de aprovação: {:.2f}%".format(indice_aprovacao))

Número de estudantes: 395
Número de features: 31
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Indice de aprovação: 67.09%


### F1 score

In [4]:
y_pred = ['yes']*n_estudantes
y_true = ['yes']*n_passed + ['no']*n_failed
score = f1_score(y_true, y_pred, pos_label='yes', average='binary')

print("F1 score: {:.4f}".format(score))

F1 score: 0.8030


### Arrumando os dados
#### Separando atributos de variavéis alvo

In [5]:
feature_cols = list(dados_estudantes.columns[:-1])
target_cols = dados_estudantes.columns[-1] 

print("Colunas de atributos:\n{}".format(feature_cols))
print("\nColuna-alvo: {}".format(target_cols))

X_all = dados_estudantes[feature_cols]
y_all = dados_estudantes[target_cols]

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed


#### Pré processamento de dados estudantis
 * Não numericos -> binários
 * Categorico    -> Variavéis Dummy

In [6]:
def preprocess_features(X):
    
    output = pd.DataFrame(index = X.index)
    for col, col_data in X.iteritems():
 
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
    
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("{} features processadas:\nTendo como resultantes:\n\n{}".format(len(X_all.columns), list(X_all.columns)))

48 features processadas:
Tendo como resultantes:

['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Divisão de treinamento e de teste

In [7]:
X_all, y_all = shuffle(X_all, y_all, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify=y_all, test_size=0.24, random_state=42)


print("O conjunto de treinamento tem {} amostras.".format(X_train.shape[0]))
print("O conjunto de teste tem {} amostras.".format(X_test.shape[0]))

print("\nVerificando balanceamento:")
print("Indice de aprovação no conjunto de treinamento: {:.2f}%".format(100 * (y_train == 'yes').mean()))
print("Indice de aprovação no conjunto de teste:       {:.2f}%".format(100 * (y_test == 'yes').mean()))

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.

Verificando balanceamento:
Indice de aprovação no conjunto de treinamento: 67.00%
Indice de aprovação no conjunto de teste:       67.37%
